# Introduction

It was the end of Ramadan when I began thinking about how I shall address the habit of night prayers (qiyyam) after the holy month, and thinking to myself that I should specifiy a daily goal (werd). And then I remembered the hadith:

قال الرسول عليه الصلاة و السلام (من قامَ بعشرِ آياتٍ لم يُكتب منَ الغافلينَ، ومن قامَ بمائةِ آيةٍ كتبَ منَ القانتينَ، ومن قامَ بألفِ آيةٍ كتبَ منَ المقنطرينَ)  الراوي: عبدالله بن عمرو | المحدث: الألباني | المصدر: صحيح أبي داود 1398 | خلاصة حكم المحدث: صحيح

The Prophet (peace be upon him) said: If anyone prays at night reciting regularly ten verses, he will not be recorded among the negligent; if anyone prays at night and recites a hundred verses, he will be recorded among those who are obedient to Allah; and if anyone prays at night reciting one thousand verses, he will be recorded among those who receive huge rewards. 

So then I contemplated how could one recite 1000 verses? I don't remember any time during Ramadan where I could have reached that number. I talked about it with my wife and she said that she knew that this was possible, it was by reading the last two Juz's of Quran that you would be able to finish the 1000 verses.

I thought that it was interesting, but I then remembered that I was a budding data scientist, and that I could easily answer this in a data-driven way. I challenged myself to find an easier way than this.

So this project is split into 4 parts:
1. Getting the data and processing it
2. Analyzing the data
3. Illustrating how to pray using either ways
4. Other Scenarios

## Getting the data

I found this amazing github repo called [quran-json](https://github.com/risan/quran-json) which has quran text in JSON format, making it really easy to get the data that I had in mind.

In [1]:
import urllib, json, re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from pandas.plotting import table 

%matplotlib inline
plt.rcParams['figure.dpi'] = 120

The information that I wanted was all surahs, their total verses, words and characters. That's why I downloaded a json file providing general info about each surah (it's number, name, translation , revelation place and total verses)

In [2]:
url = 'https://unpkg.com/quran-json@latest/json/surahs.pretty.json'
response = urllib.request.urlopen(url)
data = json.loads(response.read())
surahs = pd.DataFrame(data)
surahs.head()

,number,name,transliteration_en,translation_en,total_verses,revelation_type
0,1,سورة الفاتحة,Al-Faatiha,The Opening,7,Meccan
1,2,سورة البقرة,Al-Baqara,The Cow,286,Medinan
2,3,سورة آل عمران,Aal-i-Imraan,The Family of Imraan,200,Medinan
3,4,سورة النساء,An-Nisaa,The Women,176,Medinan
4,5,سورة المائدة,Al-Maaida,The Table,120,Medinan


I then downloaded another json file containing verses labeled by the surah they belong to, which I used to calculate the number of words and character in each surah.

In [3]:
url = 'https://unpkg.com/quran-json@latest/json/quran/text.pretty.json'
response = urllib.request.urlopen(url)
data = json.loads(response.read())
text = pd.DataFrame(data)
save_df_as_png(text, 'imgs/text.png')

text.head()

NameError: name 'save_df_as_png' is not defined

In order to count the real number of chars, we have to remove diacritics as they are considered separate characters.

In [ ]:
def remove_diacritics(text):
    arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, '', text)
    return text

In [ ]:
text['words_count'] = text.content.str.split().apply(len)
text['chars_count'] = text['content'].apply(remove_diacritics).apply(lambda x: len(x) - x.count(' '))

In [ ]:
surah_agg = text.groupby('surah_number').agg({'verse_number': len, 'chars_count': sum, 'words_count': sum})
surah_agg.head()

In [ ]:
surahs['total_words'] = surahs['number'].map(surah_agg['words_count'])
surahs['total_chars'] = surahs['number'].map(surah_agg['chars_count'])

Now that I had the data, I needed to find out how to answer the question I was posing. In order to find the easiest way to pray with 1000 verses, I needed to find which surahs have the easiest verses to read, that is they contain the least amount of words or characters per verse.

In [ ]:
surah_word_per_verse = (surah_agg['words_count']/surah_agg['verse_number']).sort_values()
surah_char_per_verse = (surah_agg['chars_count']/surah_agg['verse_number']).sort_values()
surahs['word_per_verse'] = surahs['number'].map(surah_word_per_verse)
surahs['char_per_verse'] = surahs['number'].map(surah_char_per_verse)

In [ ]:
# save datasets
surahs.to_csv('surahs.csv', index=False)
text.to_csv('text.csv', index=False)

In [ ]:
# read datasets
surahs = pd.read_csv('surahs.csv')
text = pd.read_csv('text.csv')

## Analyzing the data

The basic idea that I had was to rank the Surahs according to their word to verse, or char to verse ratio. After that I could figure out the cutoff point where the cumlative sum of the Surahs' verses reaches 1000, then sort them according to their number in the Quran to find out how to read them.

But first we need to look at the baseline, in order to compare it with the new ways we are trying to find.

### Analyzing the old way

The old way starts by Al-Mulk, and end at Al-Nas.

In [ ]:
data_old = surahs[66:]
data_old.head()

In [ ]:
print('Total number of Surahs:', data_old.shape[0])
print('Total number of Verses:', data_old.total_verses.sum())
print('Total number of Words:', data_old.total_words.sum())
print('Total number of Characters:', data_old.total_chars.sum())

Since we read Al-Fatihah at least one time in Qiyyam (In a one rakaa' wetr prayer), that would decrease 14 verses from the total 1000 verses. Meaning the we don't more than 993 verses to complete 1000 verses including the verses of Al-Fatihah.

Now that we have our baseline, let's take a look into what the data dictates.

### Finding a way by optimising word to verse ratio

In [ ]:
data_word = surahs.sort_values(['word_per_verse']).set_index('transliteration_en')
ax = sns.displot(data_word, x='word_per_verse', kind='kde', height=4, aspect=3)
# plt.xticks([])
plt.ylabel('Density', fontsize=9, fontfamily='serif')
plt.xlabel('Word to Verse Ratio', fontsize=9, fontfamily='serif')
plt.title('Distribution of Word to Verse Ratio in all Surahs of the Quran', fontsize=12, fontfamily='serif')

fig.patch.set_facecolor('white')
plt.savefig('imgs/word_to_verse.png', bbox_inches='tight')

We can see the word to verse ratio in all Surahs is bimodal. We can see that the a great proportion of the Surahs have low word to verse ratio. So how many of them would it take to complete 1000 verses if I we ranked the Surahs by word to verse in ascending order?

In [ ]:
plt.figure(figsize=(10, 3))
total_verses_cumsum = data_word.total_verses.cumsum()
total_chars_cumsum = data_word.total_chars.cumsum()
total_words_cumsum = data_word.total_words.cumsum()
last_surah_idx = np.where(np.array(total_verses_cumsum >= 1000))[0][0]
xs = np.arange(len(total_verses_cumsum))
plt.plot(xs, total_verses_cumsum);
plt.xticks([last_surah_idx], [last_surah_idx+1]);
plt.xlabel('Surahs', fontfamily='serif', fontsize=8)
plt.ylabel('Cumlative Verses', fontfamily='serif', fontsize=8)
plt.fill_between(xs[:last_surah_idx+1], total_verses_cumsum[:last_surah_idx+1]);
plt.title('Cumlative Verses over Surahs (Ascending order over word to verse)', fontsize=10, fontfamily='serif')
plt.savefig('imgs/cumlative_verses.png', bbox_inches='tight');

So using this information, we know that it would take 34 surahs to complete 1000 verses. But Surahs in here are sorted by the word to verse ratio, how would we actually read them if they were sorted by their number?

In [ ]:
first_way = data_word[:last_surah_idx+1].sort_values('number')
first_way

In [ ]:
#Set figure & Axes
fig, ax = plt.subplots(figsize=(15, 4), constrained_layout=True)
ax.set_ylim(-2, 1.75)
ax.set_xlim(0, first_way.shape[0])

# Timeline : line
ax.axhline(0, xmin=1/34, xmax=33/34, c='#4a4a4a', zorder=1)


# The surahs recited within each Rakaa
tl_sub_x = np.arange(1, 34)
tl_sub_text = [get_surah(first_way, i) for i in tl_sub_x]
tl_sub_number = first_way[1:].number 

# Highlighting surahs that come before and after skips
tl_sub_x_skips = [5, 6, 13, 14, 24, 25, 27, 28, 29, 31, 32 ,33]


# ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=120, c='#4a4a4a', zorder=2);
# ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=30, c='#fafafa', zorder=3);
ax.scatter(tl_sub_x, y=np.zeros(len(tl_sub_x)), s=30, c='#4a4a4a', zorder=2)
ax.scatter(tl_sub_x_skips, y=np.zeros(len(tl_sub_x_skips)), s=10, c='red', zorder=3)


    
o = 1
for x, date, n in zip(tl_sub_x, tl_sub_text, tl_sub_number):
    if o == 1:
        h1 = 0.13
        h2 = 0.25
    else:
        h1 = -0.2
        h2 = -0.35
    ax.text(x, h1, date, ha='center', 
            fontfamily='serif', fontweight='light',
            color='#4a4a4a',fontsize=8)
    ax.text(x, h2, n, ha='center',
            fontfamily='serif', fontweight='light',
            color='#4a4a4a', fontsize=9)
    o *= -1
    
    
# Spine
for spine in ["left", "top", "right", "bottom"]:
    ax.spines[spine].set_visible(False)
    
ax.set_yticks([])
ax.set_xticks([]);

# Legend

# Beginning of new rakaa
# white_point = [25]
# ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
# ax.scatter(white_point, [1.6], s=30, c='#fafafa', zorder=3)
# ax.text(25.5, 1.56, "Begin Reciting Surah / End Rakaa", fontweight="bold", fontfamily='serif', fontsize=8)

# Recite Surah
surah_point = [29]
# ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
ax.scatter(surah_point, [1.4], s=30, c='#4a4a4a', zorder=2)
ax.text(29.5, 1.36, "Recite Surah", fontweight="bold", fontfamily='serif', fontsize=8)

# skips (two red points)
red_points = [29]
ax.scatter(red_points, y=np.zeros(len(red_points))+1.2, s=30, c='#4a4a4a', zorder=2)
ax.scatter(red_points, y=np.zeros(len(red_points))+1.2, s=10, c='red', zorder=3)
ax.text(29.5, 1.16, "Skip Until Surah and Recite", fontweight="bold", fontfamily='serif', fontsize=8)

# Title
ax.set_title("The Easiest Way to Recite 1000 Versee", fontweight="bold", fontfamily='serif', fontsize=16, color='#4a4a4a');
ax.text(14.3,1.57,"With Correct Surahs' Order", fontfamily='serif', fontsize=12, color='#4a4a4a');

fig.patch.set_facecolor('white')
fig.savefig('imgs/first_order.png', bbox_inches='tight')

So by following this you wouldn't start at Al-Mulk, but rather at As-Saaffaat, and then skipping to Ar-Rahmaan and Al-Waaqia, then skip to Al-Muddaththir and recite until the end while skipping some Surahs that have high word to verse ratio.

In [ ]:
print('Total number of Surahs:', first_way.shape[0])
print('Total number of Verses:', first_way.total_verses.sum())
print('Total number of Words:', first_way.total_words.sum())
print('Total number of Characters:', first_way.total_chars.sum())
print()
print('Word count difference with Baseline:', data_old.total_words.sum() - first_way.total_words.sum())
print('Character count difference with Baseline:', data_old.total_chars.sum() - first_way.total_chars.sum())

To better understand how does this difference fair out, let's find out how many pages of Juz' Amma (جزء عم) this way cuts out. 

In [ ]:
juz_amma = surahs.query('number >= 78')
page_word_count = juz_amma.total_words.sum() / 23
words_saved = data_old.total_words.sum() - first_way.total_words.sum()
pages_saved = words_saved/page_word_count

print('The number of pages from Juz Amma this way saves is', pages_saved)

### Finding a way by optimising char to verse ratio

In [ ]:
data_char = surahs.sort_values(['char_per_verse']).set_index('transliteration_en')
sns.displot(data_word, x='char_per_verse', kind='kde', height=4, aspect=3)
# plt.xticks([])
plt.ylabel('Density')
plt.xlabel('Char to Verse Ratio')
plt.title('Char to Verse Ratio of different Surahs in the Quran');

In [ ]:
plt.figure(figsize=(10, 3))
total_verses_cumsum = data_char.total_verses.cumsum()
total_chars_cumsum = data_char.total_chars.cumsum()
total_words_cumsum = data_char.total_words.cumsum()
last_surah_idx = np.where(np.array(total_verses_cumsum >= 1000))[0][0]
xs = np.arange(len(total_verses_cumsum))
plt.plot(xs, total_verses_cumsum);
plt.xticks([last_surah_idx], [last_surah_idx+1]);
plt.xlabel('Surahs')
plt.ylabel('Cumlative Verses')
plt.fill_between(xs[:last_surah_idx+1], total_verses_cumsum[:last_surah_idx+1]);

So it would take 35 surahs to complete 1000 verses. Let's take a look into how they are ordered.

In [ ]:
second_way = data_word[:last_surah_idx+1].sort_values('number')
second_way

In [ ]:
print('Total number of Surahs:', second_way.shape[0])
print('Total number of Verses:', second_way.total_verses.sum())
print('Total number of Words:', second_way.total_words.sum())
print('Total number of Characters:', second_way.total_chars.sum())
print()
print('Word count difference with Baseline:', data_old.total_words.sum() - second_way.total_words.sum())
print('Character count difference with Baseline:', data_old.total_chars.sum() - second_way.total_chars.sum())

In [ ]:
words_saved = data_old.total_words.sum() - second_way.total_words.sum()
pages_saved = words_saved/page_word_count

print('The number of pages from Juz Amma this way saves is', pages_saved)

So this way turned out to be nearly identical to the first way. But the first one still trumps the second.

But this bothered me, as I didn't want to be always on the lookout for Surahs I was going to skip, whereas the old way provided a slick way to just go through the last two Juz's, was there a way to find a middle ground?


I figured out that I should stick with the most important part of the two ways, which are the parts that finish the bulk of the verses. The first 3 Surahs (As-Saaffaat, Ar-Rahmaan and Al-Waaqia) completed 356 verses, which is around 35% of the way. If we recited them and started reciting without any skips from Al-Muddaththir, when could we get the 1000 verses?

In [ ]:
rem_verses = 993 - 356
main_surahs = [1, 37, 55, 56]  # numbers of Al-Fatihah, As-Saaffaat, Ar-Rahmaan and Al-Waaqia
data = surahs.query('number in @main_surahs or number >= 74').set_index('transliteration_en')
last_surah_idx = np.where(data[4:].cumsum().total_verses >= rem_verses)[0][0]
last_surah = data.index[4:][last_surah_idx]
last_surah_number = data.number[last_surah]
third_way = data.query('number <= @last_surah_number')
third_way

In [ ]:
third_way = third_way.reset_index()

In [ ]:
#Set figure & Axes
fig, ax = plt.subplots(figsize=(15, 4), constrained_layout=True)
ax.set_ylim(-2, 1.75)
ax.set_xlim(0, third_way.shape[0])

# Timeline : line
ax.axhline(0, xmin=1/29, xmax=28/29, c='#4a4a4a', zorder=1)

# The surahs recited within each Rakaa
tl_sub_x = np.arange(1, 29)
tl_sub_text = [get_surah(third_way, i) for i in tl_sub_x]
tl_sub_number = third_way[1:].number 



# ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=120, c='#4a4a4a', zorder=2);
# ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=30, c='#fafafa', zorder=3);
ax.scatter(tl_sub_x, y=np.zeros(len(tl_sub_x)), s=30, c='#4a4a4a', zorder=2)
# ax.scatter(tl_sub_x_skips, y=np.zeros(len(tl_sub_x_skips)), s=10, c='red', zorder=3)

    
o = 1
for x, date, n in zip(tl_sub_x, tl_sub_text, tl_sub_number):
    if o == 1:
        h1 = 0.13
        h2 = 0.25
    else:
        h1 = -0.2
        h2 = -0.35
    ax.text(x, h1, date, ha='center', 
            fontfamily='serif', fontweight='light',
            color='#4a4a4a',fontsize=8)
    ax.text(x, h2, n, ha='center',
            fontfamily='serif', fontweight='light',
            color='#4a4a4a', fontsize=9)
    o *= -1
    
    
# Spine
for spine in ["left", "top", "right", "bottom"]:
    ax.spines[spine].set_visible(False)
    
ax.set_yticks([])
ax.set_xticks([]);

# Legend

# Beginning of new rakaa
# white_point = [25]
# ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
# ax.scatter(white_point, [1.6], s=30, c='#fafafa', zorder=3)
# ax.text(25.5, 1.56, "Begin Reciting Surah / End Rakaa", fontweight="bold", fontfamily='serif', fontsize=8)

# Recite Surah
surah_point = [25]
# ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
ax.scatter(surah_point, [1.4], s=30, c='#4a4a4a', zorder=2)
ax.text(25.5, 1.36, "Recite Surah", fontweight="bold", fontfamily='serif', fontsize=8)

# skips (two red points)
red_points = [25]
ax.scatter(red_points, y=np.zeros(len(red_points))+1.2, s=30, c='#4a4a4a', zorder=2)
ax.scatter(red_points, y=np.zeros(len(red_points))+1.2, s=10, c='red', zorder=3)
ax.text(25.5, 1.16, "Skip Until Surah and Recite", fontweight="bold", fontfamily='serif', fontsize=8)

# Title
ax.set_title("The Second Easiest Way to Recite 1000 Versee", fontweight="bold", fontfamily='serif', fontsize=16, color='#4a4a4a');
ax.text(12.1,1.57,"With Correct Surahs' Order", fontfamily='serif', fontsize=12, color='#4a4a4a');

fig.patch.set_facecolor('white')
fig.savefig('imgs/second_order.png', bbox_inches='tight')

In [ ]:
print('Total number of Surahs:', third_way.shape[0])
print('Total number of Verses:', third_way.total_verses.sum())
print('Total number of Words:', third_way.total_words.sum())
print('Total number of Characters:', third_way.total_chars.sum())
print()
print('Word count difference with Baseline:', data_old.total_words.sum() - third_way.total_words.sum())
print('Character count difference with Baseline:', data_old.total_chars.sum() - third_way.total_chars.sum())

In [ ]:
words_saved = data_old.total_words.sum() - third_way.total_words.sum()
pages_saved = words_saved/page_word_count

print('The number of pages from Juz Amma this way saves is', pages_saved)

By comparing the character difference to Surahs char count, we find that the difference between the baseline in the new way which is similar in simplicity to it is around 6.25 pages of Juz' Amma. 

Now there are 2 choices for, so you either go with the easiest way to read 1000 verses which saves you around 8.25 pages, or you go with a still easy way that saves you 6.25 pages, or you go with the old way by reading the two Juz's back to back.

This question wasn't intended as a call for laziness, afterall if you intended to pray with 1000 verses how dare anyone call you lazy, but it's a way to decrease any deterrents or obstacles between intending it and actually achieving it.

In [ ]:
# comparing the three ways together
x = ['Easiest way', 'Easier way', 'Old (Hard) way',]
y = [first_way.total_words.sum(), third_way.total_words.sum(), data_old.total_words.sum()]

fig, ax = plt.subplots(figsize=(5, 3))
bar_colors = ['#3dbcfa', '#fafafa', '#fafafa']
bars = ax.bar(x, y, width=0.7, edgecolor='#4a4a4a', color=bar_colors);
ax.set_ylim([4000, 5300])

for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)
    
ax.axvline(x=-0.24, ymin=(first_way.total_words.sum()-3050)/data_old.total_words.sum(),
           ymax=(data_old.total_words.sum()-500)/data_old.total_words.sum(),
           linewidth=0.7, color='darkslategray', linestyle='--')

ax.text(-0.19, 4720,"8.3 Pages", fontfamily='serif', fontsize=8, color='#4a4a4a');

ax.axvline(x=0.75, ymin=(third_way.total_words.sum()-2400)/data_old.total_words.sum(),
           ymax=(data_old.total_words.sum()-500)/data_old.total_words.sum(),
           linewidth=0.7, color='darkslategray', linestyle='--')
ax.text(0.8, 4820,"6.25 Pages", fontfamily='serif', fontsize=8, color='#4a4a4a');


ax.set_ylabel('Word Count', fontfamily='serif', fontsize=7);

ax.set_title("Word Counts of Ways to Recite 1000 Verses", fontweight="bold", fontfamily='serif', fontsize=9, color='#4a4a4a');
ax.text(0.4, 5280,"Illustrating Difference in Pages", fontfamily='serif', fontsize=7, color='#4a4a4a');

fig.patch.set_facecolor('white')
fig.savefig('imgs/word_counts.png', bbox_inches='tight')

## Illustrating how two pray using either of the two new ways

In [ ]:
first_way = first_way.reset_index()
third_way = third_way.reset_index()

In [ ]:
def get_surah(df, idx):
    return df.iloc[idx]['transliteration_en']

In [ ]:
def get_surahs_quantiles(cumsum):
    q_12 = np.where(cumsum['total_verses'] >= 125)[0][0]
    q_25 = np.where(cumsum['total_verses'] >= 250)[0][0]
    q_37 = np.where(cumsum['total_verses'] >= 375)[0][0]
    q_50 = np.where(cumsum['total_verses'] >= 500)[0][0]
    q_67 = np.where(cumsum['total_verses'] >= 675)[0][0]
    q_75 = np.where(cumsum['total_verses'] >= 750)[0][0]
    q_87 = np.where(cumsum['total_verses'] >= 875)[0][0]
    return q_12, q_25, q_37, q_50, q_67, q_75, q_87

## First illustrating how to pray using the first way as it contains some skipping

In [ ]:
cumsum = first_way.cumsum().shift(1).fillna(0)
q_12, q_25, q_37, q_50, q_67, q_75, q_87 = get_surahs_quantiles(cumsum)

### Two Rakaas version

In [ ]:
# Set figure & Axes
fig, ax = plt.subplots(figsize=(15, 4), constrained_layout=True)
ax.set_ylim(-2, 1.75)
ax.set_xlim(0, first_way.shape[0])

# Timeline : line
ax.axhline(0, xmin=1/34, xmax=33/34, c='#4a4a4a', zorder=1)


# Choosing the surahs for beginning each Rakaa
tl_x = [1, q_50]
tl_text = [f"First Rakaa\n{get_surah(first_way, 1)}",
           f"Second Rakaa\n{get_surah(first_way, q_50)}"]

# The surahs recited within each Rakaa
tl_sub_x = np.arange(1, 34)
tl_sub_text = [get_surah(first_way, i) for i in tl_sub_x]

# Highlighting surahs that come before and after skips
tl_sub_x_skips = [2, 4, 6, 14, 25, 28, 29, 32, 33]

tl_sub_text[0] = ''
# tl_sub_text[2] = ''
tl_sub_text[q_50-1] = ''


# # tl_text = [prep_arabic_text(text) for text in tl_text]


ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=120, c='#4a4a4a', zorder=2);
ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=30, c='#fafafa', zorder=3);
ax.scatter(tl_sub_x, y=np.zeros(len(tl_sub_x)), s=30, c='#4a4a4a', zorder=2)
ax.scatter(tl_sub_x_skips, y=np.zeros(len(tl_sub_x_skips)), s=5, c='red', zorder=3)


levels = [0.3, 0.3]
markerline, stemline, baseline = ax.stem(tl_x, levels, use_line_collection=True)      
plt.setp(baseline, zorder=0)
plt.setp(markerline, marker=',', color='#4a4a4a')
plt.setp(stemline, color='#4a4a4a')


# Date Text
for x, date in zip(tl_x, tl_text):
    ax.text(x, 0.45, date, ha='center', 
            fontfamily='serif', fontweight='bold',
            color='#4a4a4a',fontsize=12)
    
o = -1
for x, date in zip(tl_sub_x, tl_sub_text):
    if date:
        if o == 1:
            h = 0.13
        else:
            h = -0.2
        ax.text(x, h, date, ha='center', 
                fontfamily='serif', fontweight='light',
                color='#4a4a4a',fontsize=8)
        o *= -1
    
# # # all what's in between
# # ax.axhline(0.18, xmin=0.7, xmax=0.78, c='#4a4a4a', zorder=0)
# # ax.axhline(0.18, xmin=0.92, xmax=0.98, c='#4a4a4a', zorder=0)

# # ax.text(0.85, 0.13, 'All Surahs in between', ha='center',
# #         fontfamily='serif', fontweight='bold',
# #         color='#4a4a4a', fontsize=10)
    
# Spine
for spine in ["left", "top", "right", "bottom"]:
    ax.spines[spine].set_visible(False)
    
ax.set_yticks([])
ax.set_xticks([]);

# Legend

# Beginning of new rakaa
white_point = [29]
ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
ax.scatter(white_point, [1.6], s=30, c='#fafafa', zorder=3)
ax.text(29.5, 1.56, "Begin Reciting Surah / End Rakaa", fontweight="bold", fontfamily='serif', fontsize=8)

# Recite Surah
surah_point = [29]
# ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
ax.scatter(white_point, [1.4], s=30, c='#4a4a4a', zorder=2)
ax.text(29.5, 1.36, "Recite Surah", fontweight="bold", fontfamily='serif', fontsize=8)



# skips (two red points)
# ax.axhline(1.2, xmin=28/34, xmax=29/34, c='#4a4a4a', zorder=1)
red_points = [29]
ax.scatter(red_points, y=np.zeros(len(red_points))+1.2, s=30, c='#4a4a4a', zorder=2)
ax.scatter(red_points, y=np.zeros(len(red_points))+1.2, s=5, c='red', zorder=3)
ax.text(29.5, 1.16, "Skip Until Surah and Recite", fontweight="bold", fontfamily='serif', fontsize=8)


# Title
ax.set_title("How To Recite 1000 Verses (In Two Rakaas)", fontweight="bold", fontfamily='serif', fontsize=16, color='#4a4a4a');
ax.text(13.5,1.57,"Saving Almost 8.5 Pages of Juz Amma", fontfamily='serif', fontsize=12, color='#4a4a4a');

fig.patch.set_facecolor('white')
fig.savefig('imgs/first_two.png', bbox_inches='tight')

### Four Rakaas version

In [ ]:
# Set figure & Axes
fig, ax = plt.subplots(figsize=(15, 4), constrained_layout=True)
ax.set_ylim(-2, 1.75)
ax.set_xlim(0, first_way.shape[0])

# Timeline : line
ax.axhline(0, xmin=1/34, xmax=33/34, c='#4a4a4a', zorder=1)


# Choosing the surahs for beginning each Rakaa
tl_x = [1, q_25, q_50, q_75]
tl_text = [f"First Rakaa\n{get_surah(first_way, 1)}",
           f"Second Rakaa\n{get_surah(first_way, q_25)}",
           f"Third Rakaa\n{get_surah(first_way, q_50)}",
           f"Fourth Rakaa\n{get_surah(first_way, q_75)}"]

# The surahs recited within each Rakaa
tl_sub_x = np.arange(1, 34)
tl_sub_text = [get_surah(first_way, i) for i in tl_sub_x]

# Highlighting surahs that come before and after skips
tl_sub_x_skips = [2, 4, 6, 14, 25, 28, 29, 32, 33]

tl_sub_text[0] = ''
tl_sub_text[q_25-1] = ''
tl_sub_text[q_50-1] = ''
tl_sub_text[q_75-1] = ''


# # tl_text = [prep_arabic_text(text) for text in tl_text]


ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=120, c='#4a4a4a', zorder=2);
ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=30, c='#fafafa', zorder=3);
ax.scatter(tl_sub_x, y=np.zeros(len(tl_sub_x)), s=30, c='#4a4a4a', zorder=2)
ax.scatter(tl_sub_x_skips, y=np.zeros(len(tl_sub_x_skips)), s=10, c='red', zorder=3)


levels = [0.3 for _ in range(len(tl_x))]
levels[1] = -0.4
markerline, stemline, baseline = ax.stem(tl_x, levels, use_line_collection=True)      
plt.setp(baseline, zorder=0)
plt.setp(markerline, marker=',', color='#4a4a4a')
plt.setp(stemline, color='#4a4a4a')


# Date Text
for x, date, level in zip(tl_x, tl_text, levels):
    ax.text(x, 0.45 if level > 0 else -.8, date, ha='center', 
            fontfamily='serif', fontweight='bold',
            color='#4a4a4a',fontsize=12)
    
o = -1
for x, date in zip(tl_sub_x, tl_sub_text):
    if date:
        if o == 1:
            h = 0.13
        else:
            h = -0.2
        ax.text(x, h, date, ha='center', 
                fontfamily='serif', fontweight='light',
                color='#4a4a4a',fontsize=8)
        o *= -1
    
    
# Spine
for spine in ["left", "top", "right", "bottom"]:
    ax.spines[spine].set_visible(False)
    
ax.set_yticks([])
ax.set_xticks([]);

# Legend

# Beginning of new rakaa
white_point = [29]
ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
ax.scatter(white_point, [1.6], s=30, c='#fafafa', zorder=3)
ax.text(29.5, 1.56, "Begin Reciting Surah / End Rakaa", fontweight="bold", fontfamily='serif', fontsize=8)

# Recite Surah
surah_point = [29]
# ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
ax.scatter(white_point, [1.4], s=30, c='#4a4a4a', zorder=2)
ax.text(29.5, 1.36, "Recite Surah", fontweight="bold", fontfamily='serif', fontsize=8)



# skips (two red points)
red_points = [29]
ax.scatter(red_points, y=np.zeros(len(red_points))+1.2, s=30, c='#4a4a4a', zorder=2)
ax.scatter(red_points, y=np.zeros(len(red_points))+1.2, s=10, c='red', zorder=3)
ax.text(29.5, 1.16, "Skip Until Surah and Recite", fontweight="bold", fontfamily='serif', fontsize=8)

# Title
ax.set_title("How To Recite 1000 Verses (In Four Rakaas)", fontweight="bold", fontfamily='serif', fontsize=16, color='#4a4a4a');
ax.text(13.5,1.57,"Saving Almost 8.5 Pages of Juz Amma", fontfamily='serif', fontsize=12, color='#4a4a4a');

fig.patch.set_facecolor('white')
fig.savefig('imgs/first_four.png', bbox_inches='tight')

### Eight Rakaas version

In [ ]:
# Set figure & Axes
fig, ax = plt.subplots(figsize=(15, 4), constrained_layout=True)
fig.patch.set_facecolor('white')
ax.set_ylim(-2, 1.75)
ax.set_xlim(0, first_way.shape[0])

# Timeline : line
ax.axhline(0, xmin=1/34, xmax=33/34, c='#4a4a4a', zorder=1)


# Choosing the surahs for beginning each Rakaa
tl_x = [1, q_12, q_25, q_37, q_50, q_67, q_75, q_87]
tl_text = [f"First Rakaa\n{get_surah(first_way, 1)}",
           f"Second Rakaa\n{get_surah(first_way, q_12)}",
           f"Third Rakaa\n{get_surah(first_way, q_25)}",
           f"Fourth Rakaa\n{get_surah(first_way, q_37)}",
           f"Fifth Rakaa\n{get_surah(first_way, q_50)}",
           f"Sixth Rakaa\n{get_surah(first_way, q_67)}",
           f"Seventh Rakaa\n{get_surah(first_way, q_75)}",
           f"Eighth Rakaa\n{get_surah(first_way, q_87)}"]

# The surahs recited within each Rakaa
tl_sub_x = np.arange(1, 34)
tl_sub_text = [get_surah(first_way, i) for i in tl_sub_x]

# Highlighting surahs that come before and after skips
tl_sub_x_skips = [2, 4, 6, 14, 25, 28, 29, 32, 33]

tl_sub_text[0] = ''
tl_sub_text[q_12-1] = ''
tl_sub_text[q_25-1] = ''
tl_sub_text[q_37-1] = ''
tl_sub_text[q_50-1] = ''
tl_sub_text[q_67-1] = ''
tl_sub_text[q_75-1] = ''
tl_sub_text[q_87-1] = ''


# # tl_text = [prep_arabic_text(text) for text in tl_text]


ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=120, c='#4a4a4a', zorder=2);
ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=40, c='#fafafa', zorder=3);
ax.scatter(tl_sub_x, y=np.zeros(len(tl_sub_x)), s=40, c='#4a4a4a', zorder=2)
ax.scatter(tl_sub_x_skips, y=np.zeros(len(tl_sub_x_skips)), s=10, c='red', zorder=3)


levels = [0.3 for _ in range(len(tl_x))]
levels[1] = -0.4
levels[2] = 0.75
levels[3] = 1.2
levels[6] = -0.4

markerline, stemline, baseline = ax.stem(tl_x, levels, use_line_collection=True)      
plt.setp(baseline, zorder=0)
plt.setp(markerline, marker=',', color='#4a4a4a')
plt.setp(stemline, color='#4a4a4a')


# Date Text
for x, date, level in zip(tl_x, tl_text, levels):
    ax.text(x, level+0.15 if level > 0 else -.8, date, ha='center', 
            fontfamily='serif', fontweight='bold',
            color='#4a4a4a',fontsize=12)
    
o = -1
for x, date in zip(tl_sub_x, tl_sub_text):
    if date:
        if o == 1:
            h = 0.13
        else:
            h = -0.2
        ax.text(x, h, date, ha='center', 
                fontfamily='serif', fontweight='light',
                color='#4a4a4a',fontsize=8)
        o *= -1
    
# Spine
for spine in ["left", "top", "right", "bottom"]:
    ax.spines[spine].set_visible(False)
    
ax.set_yticks([])
ax.set_xticks([]);

# Legend

# Beginning of new rakaa
white_point = [29]
ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
ax.scatter(white_point, [1.6], s=30, c='#fafafa', zorder=3)
ax.text(29.5, 1.56, "Begin Reciting Surah / End Rakaa", fontweight="bold", fontfamily='serif', fontsize=8)

# Recite Surah
surah_point = [29]
# ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
ax.scatter(white_point, [1.4], s=30, c='#4a4a4a', zorder=2)
ax.text(29.5, 1.36, "Recite Surah", fontweight="bold", fontfamily='serif', fontsize=8)



# skips (two red points)
red_points = [29]
ax.scatter(red_points, y=np.zeros(len(red_points))+1.2, s=30, c='#4a4a4a', zorder=2)
ax.scatter(red_points, y=np.zeros(len(red_points))+1.2, s=10, c='red', zorder=3)
ax.text(29.5, 1.16, "Skip Until Surah and Recite", fontweight="bold", fontfamily='serif', fontsize=8)

# Title
ax.set_title("How To Recite 1000 Verses (In Eight Rakaas)", fontweight="bold", fontfamily='serif', fontsize=16, color='#4a4a4a');
ax.text(13.5,1.57,"Saving Almost 8.5 Pages of Juz Amma", fontfamily='serif', fontsize=12, color='#4a4a4a');

fig.patch.set_facecolor('white')
fig.savefig('imgs/first_eight.png', bbox_inches='tight')

### Two Rakaa Version

In [ ]:
from bidi.algorithm import get_display
import arabic_reshaper


def prep_arabic_text(text):
    reshaped_text = arabic_reshaper.reshape(u"{}".format(text))
    artext = get_display(reshaped_text)
    return artext

# Set figure & Axes
fig, ax = plt.subplots(figsize=(15, 4), constrained_layout=True)
ax.set_ylim(-2, 1.75)
ax.set_xlim(0, 29)

# Timeline : line
ax.axhline(0, xmin=1/29, xmax=28/29, c='#4a4a4a', zorder=1)

# Choosing the surahs for beginning each Rakaa
tl_x = [1, q_50]
tl_text = [f"First Rakaa\n{get_surah(first_way, 1)}",
           f"Second Rakaa\n{get_surah(first_way, q_50)}"]

# The surahs recited within each Rakaa
tl_sub_x = np.arange(1, 29)
tl_sub_text = [get_surah(first_way, i) for i in tl_sub_x]

tl_sub_text[0] = ''
tl_sub_text[q_50-1] = ''


ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=120, c='#4a4a4a', zorder=2);
ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=30, c='#fafafa', zorder=3);
ax.scatter(tl_sub_x, y=np.zeros(len(tl_sub_x)), s=30, c='#4a4a4a', zorder=2)

levels = [0.3 for _ in range(len(tl_x))]
markerline, stemline, baseline = ax.stem(tl_x, levels, use_line_collection=True)      
plt.setp(baseline, zorder=0)
plt.setp(markerline, marker=',', color='#4a4a4a')
plt.setp(stemline, color='#4a4a4a')


# Date Text
for x, date, level in zip(tl_x, tl_text, levels):
    ax.text(x, level+0.15 if level > 0 else -.8, date, ha='center', 
            fontfamily='serif', fontweight='bold',
            color='#4a4a4a',fontsize=12)
    
o = 1
for x, date in zip(tl_sub_x, tl_sub_text):
    if o == 1:
        h = 0.13
    else:
        h = -0.2
    ax.text(x, h, date, ha='center', 
            fontfamily='serif', fontweight='light',
            color='#4a4a4a',fontsize=9)
    o *= -1
    
    
# Spine
for spine in ["left", "top", "right", "bottom"]:
    ax.spines[spine].set_visible(False)
    
ax.set_yticks([])
ax.set_xticks([]);

# Legend

# Beginning of new rakaa
white_point = [25]
ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
ax.scatter(white_point, [1.6], s=30, c='#fafafa', zorder=3)
ax.text(25.5, 1.56, "Begin Reciting Surah / End Rakaa", fontweight="bold", fontfamily='serif', fontsize=8)

# Recite Surah
surah_point = [25]
# ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
ax.scatter(white_point, [1.4], s=30, c='#4a4a4a', zorder=2)
ax.text(25.5, 1.36, "Recite Surah", fontweight="bold", fontfamily='serif', fontsize=8)

# Title
ax.set_title("How to recite 1000 verses (In Two Rakaas)", fontweight="bold", fontfamily='serif', fontsize=16, color='#4a4a4a');
ax.text(10.9,1.57,"Saving Almost 6.3 Pages From Juz Amma", fontfamily='serif', fontsize=12, color='#4a4a4a');

fig.patch.set_facecolor('white')
fig.savefig('imgs/second_two.png', bbox_inches='tight')

### Four Rakaas Version

In [ ]:
# Set figure & Axes
fig, ax = plt.subplots(figsize=(15, 4), constrained_layout=True)
ax.set_ylim(-2, 1.75)
ax.set_xlim(0, 29)

# Timeline : line
ax.axhline(0, xmin=1/29, xmax=28/29, c='#4a4a4a', zorder=1)

# Choosing the surahs for beginning each Rakaa
tl_x = [1, q_25, q_50, q_75]
tl_text = [f"First Rakaa\n{get_surah(first_way, 1)}",
           f"Second Rakaa\n{get_surah(first_way, q_25)}",
           f"Third Rakaa\n{get_surah(first_way, q_50)}",
           f"Fourth Rakaa\n{get_surah(first_way, q_75)}"]

# The surahs recited within each Rakaa
tl_sub_x = np.arange(1, 29)
tl_sub_text = [get_surah(first_way, i) for i in tl_sub_x]

# # Highlighting surahs that come before and after skips
# tl_sub_x_skips = [5, 6, 13, 14, 24, 25, 27, 28, 29, 31, 32 ,33]

tl_sub_text[0] = ''
tl_sub_text[q_25-1] = ''
tl_sub_text[q_50-1] = ''
tl_sub_text[q_75-1] = ''


ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=120, c='#4a4a4a', zorder=2);
ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=30, c='#fafafa', zorder=3);
ax.scatter(tl_sub_x, y=np.zeros(len(tl_sub_x)), s=30, c='#4a4a4a', zorder=2)

levels = [0.3 for _ in range(len(tl_x))]
levels[1] = 0.75
markerline, stemline, baseline = ax.stem(tl_x, levels, use_line_collection=True)      
plt.setp(baseline, zorder=0)
plt.setp(markerline, marker=',', color='#4a4a4a')
plt.setp(stemline, color='#4a4a4a')


# Date Text
for x, date, level in zip(tl_x, tl_text, levels):
    ax.text(x, level+0.15 if level > 0 else -.8, date, ha='center', 
            fontfamily='serif', fontweight='bold',
            color='#4a4a4a',fontsize=12)
    
o = 1
for x, date in zip(tl_sub_x, tl_sub_text):
    if o == 1:
        h = 0.13
    else:
        h = -0.2
    ax.text(x, h, date, ha='center', 
            fontfamily='serif', fontweight='light',
            color='#4a4a4a',fontsize=8)
    o *= -1
    
    
# Spine
for spine in ["left", "top", "right", "bottom"]:
    ax.spines[spine].set_visible(False)
    
ax.set_yticks([])
ax.set_xticks([]);

# Legend

# Beginning of new rakaa
white_point = [25]
ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
ax.scatter(white_point, [1.6], s=30, c='#fafafa', zorder=3)
ax.text(25.5, 1.56, "Begin Reciting Surah / End Rakaa", fontweight="bold", fontfamily='serif', fontsize=8)

# Recite Surah
surah_point = [25]
# ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
ax.scatter(white_point, [1.4], s=30, c='#4a4a4a', zorder=2)
ax.text(25.5, 1.36, "Recite Surah", fontweight="bold", fontfamily='serif', fontsize=8)

# Title
ax.set_title("How to recite 1000 verses (In Four Rakaas)", fontweight="bold", fontfamily='serif', fontsize=16, color='#4a4a4a');
ax.text(10.8,1.57,"Saving Almost 6.25 Pages From Juz Amma", fontfamily='serif', fontsize=12, color='#4a4a4a');

fig.patch.set_facecolor('white')
fig.savefig('imgs/second_four.png', bbox_inches='tight')

### Eight Rakaas Version

In [ ]:
# Set figure & Axes
fig, ax = plt.subplots(figsize=(15, 4), constrained_layout=True)
ax.set_ylim(-2, 1.75)
ax.set_xlim(0, 29)

# Timeline : line
ax.axhline(0, xmin=1/29, xmax=28/29, c='#4a4a4a', zorder=1)

## Choosing the surahs for beginning each Rakaa
tl_x = [1, q_12, q_25, q_37, q_50, q_67, q_75, q_87]
tl_text = [f"First Rakaa\n{get_surah(first_way, 1)}",
           f"Second Rakaa\n{get_surah(first_way, q_12)}",
           f"Third Rakaa\n{get_surah(first_way, q_25)}",
           f"Fourth Rakaa\n{get_surah(first_way, q_37)}",
           f"Fifth Rakaa\n{get_surah(first_way, q_50)}",
           f"Sixth Rakaa\n{get_surah(first_way, q_67)}",
           f"Seventh Rakaa\n{get_surah(first_way, q_75)}",
           f"Eighth Rakaa\n{get_surah(first_way, q_87)}"]

# The surahs recited within each Rakaa
tl_sub_x = np.arange(1, 29)
tl_sub_text = [get_surah(first_way, i) for i in tl_sub_x]

# Highlighting surahs that come before and after skips
tl_sub_x_skips = [5, 6, 13, 14, 24, 25, 27, 28, 29, 31, 32 ,33]

tl_sub_text[0] = ''
tl_sub_text[q_12-1] = ''
tl_sub_text[q_25-1] = ''
tl_sub_text[q_37-1] = ''
tl_sub_text[q_50-1] = ''
tl_sub_text[q_67-1] = ''
tl_sub_text[q_75-1] = ''
tl_sub_text[q_87-1] = ''


ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=120, c='#4a4a4a', zorder=2);
ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=30, c='#fafafa', zorder=3);
ax.scatter(tl_sub_x, y=np.zeros(len(tl_sub_x)), s=30, c='#4a4a4a', zorder=2)

levels = [0.3 for _ in range(len(tl_x))]
levels[1] = -0.4
levels[2] = 0.75
levels[3] = 1.2
levels[6] = -0.4
markerline, stemline, baseline = ax.stem(tl_x, levels, use_line_collection=True)      
plt.setp(baseline, zorder=0)
plt.setp(markerline, marker=',', color='#4a4a4a')
plt.setp(stemline, color='#4a4a4a')


# Date Text
for x, date, level in zip(tl_x, tl_text, levels):
    ax.text(x, level+0.15 if level > 0 else -.8, date, ha='center', 
            fontfamily='serif', fontweight='bold',
            color='#4a4a4a',fontsize=12)
    
o = 1
for x, date in zip(tl_sub_x, tl_sub_text):
    if o == 1:
        h = 0.13
    else:
        h = -0.2
    ax.text(x, h, date, ha='center', 
            fontfamily='serif', fontweight='light',
            color='#4a4a4a',fontsize=8)
    o *= -1
    
    
# Spine
for spine in ["left", "top", "right", "bottom"]:
    ax.spines[spine].set_visible(False)
    
ax.set_yticks([])
ax.set_xticks([]);

# Legend

# Beginning of new rakaa
white_point = [25]
ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
ax.scatter(white_point, [1.6], s=30, c='#fafafa', zorder=3)
ax.text(25.5, 1.56, "Begin Reciting Surah / End Rakaa", fontweight="bold", fontfamily='serif', fontsize=8)

# Recite Surah
surah_point = [25]
# ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
ax.scatter(white_point, [1.4], s=30, c='#4a4a4a', zorder=2)
ax.text(25.5, 1.36, "Recite Surah", fontweight="bold", fontfamily='serif', fontsize=8)

# Title
ax.set_title("How to recite 1000 verses (In Eight Rakaas)", fontweight="bold", fontfamily='serif', fontsize=16, color='#4a4a4a');
ax.text(10.8,1.57,"Saving Almost 6.25 Pages From Juz Amma", fontfamily='serif', fontsize=12, color='#4a4a4a');

fig.patch.set_facecolor('white')
fig.savefig('imgs/second_eight.png', bbox_inches='tight')

## Other Scenarios

Now you might have started reading somewhere else, and you want to continue reciting a specific number of verses, but you are still looking for an easy way to do it. That's why I've the following chart.

In [ ]:
first_to_last_cumsum = first_way[1:].cumsum()
last_to_first_cumsum = first_way[1:][::-1].cumsum()

In [ ]:
#Set figure & Axes
fig, ax = plt.subplots(figsize=(15, 4), constrained_layout=True)
ax.set_ylim(-2, 1.75)
ax.set_xlim(0, first_way.shape[0])

# Timeline : line
ax.axhline(0, xmin=1/34, xmax=33/34, c='#4a4a4a', zorder=1)


# The surahs recited within each Rakaa
tl_sub_x = np.arange(1, 34)
tl_sub_text = [get_surah(first_way, i) for i in tl_sub_x]
tl_sub_cumsum = first_to_last_cumsum.total_verses.to_list()

# Highlighting surahs that come before and after skips
tl_sub_x_skips = [5, 6, 13, 14, 24, 25, 27, 28, 29, 31, 32 ,33]


# ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=120, c='#4a4a4a', zorder=2);
# ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=30, c='#fafafa', zorder=3);
ax.scatter(tl_sub_x, y=np.zeros(len(tl_sub_x)), s=30, c='#4a4a4a', zorder=2)
ax.scatter(tl_sub_x_skips, y=np.zeros(len(tl_sub_x_skips)), s=10, c='red', zorder=3)


    
o = 1
for x, date, cumsum in zip(tl_sub_x, tl_sub_text, tl_sub_cumsum):
    if o == 1:
        h1 = 0.13
        h2 = 0.25
    else:
        h1 = -0.2
        h2 = -0.35
    ax.text(x, h1, date, ha='center', 
            fontfamily='serif', fontweight='light',
            color='#4a4a4a',fontsize=8)
    ax.text(x, h2, cumsum, ha='center',
            fontfamily='serif', fontweight='light',
            color='#4a4a4a', fontsize=9)
    o *= -1
    
    
# Spine
for spine in ["left", "top", "right", "bottom"]:
    ax.spines[spine].set_visible(False)
    
ax.set_yticks([])
ax.set_xticks([]);

# Legend

# Beginning of new rakaa
# white_point = [25]
# ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
# ax.scatter(white_point, [1.6], s=30, c='#fafafa', zorder=3)
# ax.text(25.5, 1.56, "Begin Reciting Surah / End Rakaa", fontweight="bold", fontfamily='serif', fontsize=8)

# Recite Surah
surah_point = [29]
# ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
ax.scatter(surah_point, [1.4], s=30, c='#4a4a4a', zorder=2)
ax.text(29.5, 1.36, "Recite Surah", fontweight="bold", fontfamily='serif', fontsize=8)

# skips (two red points)
red_points = [29]
ax.scatter(red_points, y=np.zeros(len(red_points))+1.2, s=30, c='#4a4a4a', zorder=2)
ax.scatter(red_points, y=np.zeros(len(red_points))+1.2, s=10, c='red', zorder=3)
ax.text(29.5, 1.16, "Skip Until Surah and Recite", fontweight="bold", fontfamily='serif', fontsize=8)

# Title
ax.set_title("Cumlative Total of Verses Starting From As-Saffaat", fontweight="bold", fontfamily='serif', fontsize=16, color='#4a4a4a');
ax.text(9.45,1.57,"If you want to start from the beginning and stop at a certain number of verses", fontfamily='serif', fontsize=12, color='#4a4a4a');

fig.patch.set_facecolor('white')
fig.savefig('imgs/other_1.png', bbox_inches='tight')

In [ ]:
#Set figure & Axes
fig, ax = plt.subplots(figsize=(15, 4), constrained_layout=True)
ax.set_ylim(-2, 1.75)
ax.set_xlim(0, first_way.shape[0])

# Timeline : line
ax.axhline(0, xmin=1/34, xmax=33/34, c='#4a4a4a', zorder=1)


# The surahs recited within each Rakaa
tl_sub_x = np.arange(1, 34)
tl_sub_text = [get_surah(first_way, i) for i in tl_sub_x]
tl_sub_cumsum = last_to_first_cumsum.total_verses[::-1].to_list()

# Highlighting surahs that come before and after skips
tl_sub_x_skips = [5, 6, 13, 14, 24, 25, 27, 28, 29, 31, 32 ,33]



# ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=120, c='#4a4a4a', zorder=2);
# ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=30, c='#fafafa', zorder=3);
ax.scatter(tl_sub_x, y=np.zeros(len(tl_sub_x)), s=30, c='#4a4a4a', zorder=2)
ax.scatter(tl_sub_x_skips, y=np.zeros(len(tl_sub_x_skips)), s=10, c='red', zorder=3)


    
o = 1
for x, date, cumsum in zip(tl_sub_x, tl_sub_text, tl_sub_cumsum):
    if o == 1:
        h1 = 0.13
        h2 = 0.25
    else:
        h1 = -0.2
        h2 = -0.35
    ax.text(x, h1, date, ha='center', 
            fontfamily='serif', fontweight='light',
            color='#4a4a4a',fontsize=8)
    ax.text(x, h2, cumsum, ha='center',
            fontfamily='serif', fontweight='light',
            color='#4a4a4a', fontsize=9)
    o *= -1
    
    
# Spine
for spine in ["left", "top", "right", "bottom"]:
    ax.spines[spine].set_visible(False)
    
ax.set_yticks([])
ax.set_xticks([]);

# Legend

# Beginning of new rakaa
# white_point = [25]
# ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
# ax.scatter(white_point, [1.6], s=30, c='#fafafa', zorder=3)
# ax.text(25.5, 1.56, "Begin Reciting Surah / End Rakaa", fontweight="bold", fontfamily='serif', fontsize=8)

# Recite Surah
surah_point = [29]
# ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
ax.scatter(surah_point, [1.4], s=30, c='#4a4a4a', zorder=2)
ax.text(29.5, 1.36, "Recite Surah", fontweight="bold", fontfamily='serif', fontsize=8)

# skips (two red points)
red_points = [29]
ax.scatter(red_points, y=np.zeros(len(red_points))+1.2, s=30, c='#4a4a4a', zorder=2)
ax.scatter(red_points, y=np.zeros(len(red_points))+1.2, s=10, c='red', zorder=3)
ax.text(29.5, 1.16, "Skip Until Surah and Recite", fontweight="bold", fontfamily='serif', fontsize=8)

# Title
ax.set_title("Cumlative Total of Verses in Reverse", fontweight="bold", fontfamily='serif', fontsize=16, color='#4a4a4a');
ax.text(10,1.57,"If you want to start from somewhere on the line and recite untill the end", fontfamily='serif', fontsize=12, color='#4a4a4a');

fig.patch.set_facecolor('white')
fig.savefig('imgs/other_2.png', bbox_inches='tight')

In [ ]:
#Set figure & Axes
fig, ax = plt.subplots(figsize=(15, 4), constrained_layout=True)
ax.set_ylim(-2, 1.75)
ax.set_xlim(0, first_way.shape[0])

# Timeline : line
ax.axhline(0, xmin=1/34, xmax=33/34, c='#4a4a4a', zorder=1)


# The surahs recited within each Rakaa
tl_sub_x = np.arange(1, 34)
tl_sub_text = [get_surah(first_way, i) for i in tl_sub_x]
tl_sub_cumsum =  first_way[1:].total_verses.to_list()

# Highlighting surahs that come before and after skips
tl_sub_x_skips = [5, 6, 13, 14, 24, 25, 27, 28, 29, 31, 32 ,33]


# ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=120, c='#4a4a4a', zorder=2);
# ax.scatter(tl_x, y=np.zeros(len(tl_x)), s=30, c='#fafafa', zorder=3);
ax.scatter(tl_sub_x, y=np.zeros(len(tl_sub_x)), s=30, c='#4a4a4a', zorder=2)
ax.scatter(tl_sub_x_skips, y=np.zeros(len(tl_sub_x_skips)), s=10, c='red', zorder=3)

    
o = 1
for x, date, cumsum in zip(tl_sub_x, tl_sub_text, tl_sub_cumsum):
    if o == 1:
        h1 = 0.13
        h2 = 0.25
    else:
        h1 = -0.2
        h2 = -0.35
    ax.text(x, h1, date, ha='center', 
            fontfamily='serif', fontweight='light',
            color='#4a4a4a',fontsize=8)
    ax.text(x, h2, cumsum, ha='center',
            fontfamily='serif', fontweight='light',
            color='#4a4a4a', fontsize=9)
    o *= -1
    
    
# Spine
for spine in ["left", "top", "right", "bottom"]:
    ax.spines[spine].set_visible(False)
    
ax.set_yticks([])
ax.set_xticks([]);

# Legend

# Beginning of new rakaa
# white_point = [25]
# ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
# ax.scatter(white_point, [1.6], s=30, c='#fafafa', zorder=3)
# ax.text(25.5, 1.56, "Begin Reciting Surah / End Rakaa", fontweight="bold", fontfamily='serif', fontsize=8)

# Recite Surah
surah_point = [29]
# ax.scatter(white_point, [1.6], s=120, c='#4a4a4a', zorder=2)
ax.scatter(surah_point, [1.4], s=30, c='#4a4a4a', zorder=2)
ax.text(29.5, 1.36, "Recite Surah", fontweight="bold", fontfamily='serif', fontsize=8)

# skips (two red points)
red_points = [29]
ax.scatter(red_points, y=np.zeros(len(red_points))+1.2, s=30, c='#4a4a4a', zorder=2)
ax.scatter(red_points, y=np.zeros(len(red_points))+1.2, s=10, c='red', zorder=3)
ax.text(29.5, 1.16, "Skip Until Surah and Recite", fontweight="bold", fontfamily='serif', fontsize=8)

# Title
ax.set_title("Number of Verses in each Surah", fontweight="bold", fontfamily='serif', fontsize=16, color='#4a4a4a');
ax.text(13,1.57,"If you want to mix and match as you like", fontfamily='serif', fontsize=12, color='#4a4a4a');

fig.patch.set_facecolor('white')
fig.savefig('imgs/other_3.png', bbox_inches='tight')

## Conclusion

Now you have all the tools to recite those 1000 verses, I pray that we're all blessed to be from those who did this.

I hope that this notebook inspires those who read it to work on projects that they find interesting, and ask questions that can possibly optimize whatever good deeds they maybe doing.